# Porting Yao.jl with QuantumInformation.jl
### GiggleLiu

# overview

 [`Yao`](https://github.com/QuantumBFS/Yao.jl) is a powerful tool for quantum circuit based simulation, but it does not support many density matrix related operations. This is why we need to port `Yao.jl` with [`QuantumInformation (QI)`](https://github.com/QuantumBFS/QuantumInformation.jl) sometimes (e.g. for computing entanglement entropy).
 
* `Yao.jl` Documentation: https://quantumbfs.github.io/Yao.jl/latest/ (paper is comming out)
* `QuantumInformation.jl` paper: https://arxiv.org/abs/1806.11464
     
### `Yao` provides
* high performance quantum circuit based simulation
    * parameter management
    * gradients
    * batched regiser
* operator matrix representation and arithmatics
* [quantum algorithms](https://github.com/QuantumBFS/QuAlgorithmZoo.jl)
* [GPU support](https://github.com/QuantumBFS/CuYao.jl)

### `QI` provides

* Compute entropy from density matrices
* Quantum channels, four types of channel representations
    * Kraus Operator
    * Super operator
    * Dynamic matrices
    * Stinespring representation
* Compute norm, distance and distingushability between "states" (density matrices)
    * Hilbert–Schmidt norm and distance
    * trace norm and *distance*
    * diamond norm
    * Bures distane and Bures angles
    * *fidelity* and superfidelity
    * KL-divergence
    * JS-distance
* Compute the amount of entanglement
     * negativity
     * positive partial trace
     * concurrence
* POVM measurements

In [16]:
import Yao
using Yao: ArrayReg, ρ, mat, ConstGate, purify, exchange_sysenv, @bit_str, statevec
import QuantumInformation; const QI = QuantumInformation
using QuantumInformation: ket
using LinearAlgebra
using Test

Obtain reduced density matrices in Yao
-------------------------
The memory layout of `Yao` register and `QI` ket are similar, their basis are both [little endian](https://en.wikipedia.org/wiki/Endianness), despite they have different representation powers

* `Yao` support batch,
* `QI` is not limited to qubits.


`Yao` does not have much operations defined on density matrices, but purified states with environment,
On the other side, most operations in `QI` are defined on **(density) matrices**, they can be easily obtained in `Yao`.

In [17]:
# construct a product state, notice the indexing in `QI` starts from `1`
@test QI.ket(3, 1<<4) ≈ statevec(ArrayReg(bit"0010"))

# join two registers, notice little endian convension is used here.
reg = join(ArrayReg(bit"10"), ArrayReg(bit"11"))
v = QI.:⊗(QI.ket(0b10+1,1<<2), QI.ket(0b11+1,1<<2))
@test statevec(reg) ≈ v

Test Passed

In [18]:
# convert a Yao register to density matrix in QI
reg2dm(reg::ArrayReg{1}) = reg |> ρ |> Matrix

# e.g. obtain a reduced denstiy matrix for subsystem 1,2,3,4
reg = Yao.rand_state(10)
freg = Yao.focus!(reg, 1:4) # make qubits 1-4 active
reg2dm(freg)

16×16 Array{Complex{Float64},2}:
   0.0668399+0.0im         …    0.0048149+0.00800254im 
 -0.00683079+0.00430075im      0.00271044+0.013467im   
 -0.00405524-0.00233655im      0.00489161-0.00506099im 
   0.0041184-0.00690317im     -0.00724508+0.00433365im 
 0.000248112-0.00614303im     -0.00169715-0.0060107im  
 -0.00638715+0.00343611im  …  -0.00346919+0.0104737im  
  -0.0032589-0.00594789im     -0.00502371+0.00889227im 
   0.0053714-0.00448422im     0.000149836+0.00490488im 
 -0.00485418+0.00190183im     -0.00707738-0.0117206im  
 -0.00185245-0.0113168im      -0.00100021+0.00456715im 
 0.000202351+0.00648573im  …   9.29962e-5-0.00362312im 
   0.0038004-0.00408768im      0.00290617+0.0109155im  
 -0.00488166-0.00699333im     -0.00471523+0.000137239im
  0.00485705+0.00532262im      0.00956895-0.00457732im 
  0.00756613-0.00569826im       0.0032851+0.0014402im  
   0.0048149-0.00800254im  …    0.0786938+0.0im        

One can also convert a density matrix to a a quantum state through **purification**

In [19]:
# e.g. purify a state and recover it
reg = Yao.rand_state(6) |> Yao.focus!(1:4)
reg_p = purify(reg |> ρ; nbit_env=2)
@test Yao.fidelity(reg, reg_p)[] ≈ 1

Test Passed

entanglement & state distance
----------------


In [20]:
reg1 = Yao.rand_state(10)
freg1 = Yao.focus!(reg1, 1:4)
reg2 = Yao.rand_state(6)
freg2 = Yao.focus!(reg2, 1:4)
dm1, dm2 = freg1 |> reg2dm, freg2 |> reg2dm

# trace distance between two registers (different by a factor 2)
@test Yao.tracedist(freg1, freg2)[]/2 ≈ QI.trace_distance(dm1, dm2)

Test Passed

In [21]:
# get the entanglement entropy between system and env
@show QI.vonneumann_entropy(dm1)
@show QI.vonneumann_entropy(dm2)

QI.vonneumann_entropy(dm1) = 2.6568839293081608
QI.vonneumann_entropy(dm2) = 1.3245543916726097


1.3245543916726097

In [22]:
# KL-divergence (or relative entropy)
QI.kl_divergence(dm2, dm1)

1.5694621854109723

Note: you can defined many distances and entropies in a similar way, we don't enumerate it.

Quantum Operations/Quantum Gates
------------------------

A quantum gate in `Yao` is equivalent to a unitary channel in `QI`, matrix representations of blocks in `Yao` can be used to construct channels.

In [23]:
# construct a Kraus Operator
QI.KrausOperators([Matrix(ConstGate.P0), Matrix(ConstGate.P1), Matrix(ConstGate.Pu)])

QuantumInformation.KrausOperators{Array{Complex{Float64},2}}
    dimensions: (2, 2)
    Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
    Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]
    Complex{Float64}[0.0 + 0.0im 1.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

In [24]:
# applying a rotation gate
b1 = Yao.put(2,2=>Yao.Rx(0.3π))
c1 = QI.UnitaryChannel(mat(b1))
b2 = Yao.put(2,2=>Yao.Ry(0.3π))
c2 = QI.UnitaryChannel(mat(b2))

reg = Yao.rand_state(2)
@test copy(reg) |> b1 |> reg2dm ≈ c1(reg |> reg2dm)
:@test copy(reg) |> Yao.chain(b1,b2) |> reg2dm ≈ (c2∘c1)(reg |> reg2dm)

:(#= In[24]:9 =# @test (copy(reg) |> Yao.chain(b1, b2)) |> reg2dm ≈ (c2 ∘ c1)(reg |> reg2dm))